# $\mathrm{H} \rightarrow \mathrm{ZZ}$ : Entdeckung des Higgs-Bosons
---------------------------------------
von Artur Monsch und Artur Gottmann

Zuletzt aktualisiert am 18. Dezember 2020

-----------------------------------

Im Jahr 2012 wurde am CERN das bereits vorhergesagte Higgs-Boson entdeckt und damit eine Bestätigung des Standardmodells der Teilchenphysik erreicht. Einer der Zerfallskanäle, die zur Entdeckung führten, war der Zerfall in vier Leptonen. Dieser ist im Vergleich zu den anderen Zerfallskanälen ideal für die Analyse geeignet, die Sie nun in Form dieses Notebooks durchführen können.

Das Notebook ist der erste Teil dieser Analyse und beschäftigt sich hauptsächlich mit den simulierten und gemessenen Datensätzen. Ziel ist es, die Sensitivität zu erhöhen und ein hohes Verhältnis zwischen dem Untergrund und dem Signal in den simulierten Datensätzen zu erreichen. Am Ende der Aufgabe soll die Signifikanz an der durchgeführten Messung abgeschätzt werden. Anhand dieser Signifikanz kann eine erste Aussage über den Nachweis des Higgs-Bosons getroffen werden. Eine detaillierte statistische Behandlung der Signifikanz bis hin zur Kombination von Messungen zur Erhöhung der Signifikanz wird im zweiten Teil vorgestellt.

Als Inspiration für diese Übung kann die folgende [Beispielanalyse](http://opendata.cern.ch/record/5500) aufgeführt werden.

In [ ]:
import sys
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

sys.path.append("..")

from include.RandomHelper import check_data_state

# Check if all folders in the directory are present and unpacking them
check_data_state(directory="../data") 

# Ereignisbilder
------------------

Zum Kennenlernen der Signaturen von Proton-Proton-Kollisionsereignissen, die vom CMS-Experiment aufgezeichnet wurden, können Sie die Visualisierung des CMS-Detektors zusammen mit den Beispielen der aufgezeichneten
Ereignissen mit dem [**Ispy-WebGL-Webinterface**](https://github.com/cms-outreach/ispy-webgl) betrachten. Die verschiedenen Komponenten des CMS-Mehrzweck-Detektors, die zum Nachweis verschiedener Teilchen verwendet werden, sind in dem [**ausgewählten Detektorschnitt**](https://cds.cern.ch/record/2120661/files/CMSslice_whiteBackground.png?subformat=icon-1440) dargestellt:
<center><img src="https://cds.cern.ch/record/2120661/files/CMSslice_whiteBackground.png?subformat=icon-1440" width=75%></center>


Bei der Verwendung von Ispy-WebGL können Sie sich mit der Funktionalität der einzelnen Komponenten vertraut machen, indem Sie sie diese im Menü 'Detector' aktivieren und Ereignisse mit interessanten Signaturen aus den Ereignissammlungen auswählen. Versuchen Sie auch, 'Physics-Objects' im entsprechenden Menü einzuschalten.


<div class="alert alert-info">
Wählen Sie für jeden der unten aufgeführten Zerfälle zwei Beispiele aus und speichern Sie diese als Bild. Wie sähe ein typisches Signal dieser einzelnen Zerfälle im Detektor aus?

- $\mathrm{H} \rightarrow \mathrm{ZZ} \rightarrow 4\ell$
- $\mathrm{H} \rightarrow \gamma \gamma$
- $\mathrm{H} \rightarrow \mathrm{W}^+ \mathrm{W}^- \rightarrow 2\ell 2\nu$

</div>


Die Einschaltfunktionen einzelner Detektorkomponenten und die Möglichkeit, die einzelnen Ereignisse als Bilder zu speichern, können innerhalb der Benutzeroberfläche durchgeführt werden. Um die Ereignissammlung zu öffnen, müssen Sie zunächst die Datei `Event_collection.ig` aus dem Ordner `./data/for_event_display/ig_files/` lokal herunterladen. Anschließend kann innerhalb der Benutzeroberfläche unter dem Reiter "Öffnen" - Ordnersymbol - diese Datei (lokale Datei öffnen) ausgewählt und geöffnet werden. Die Bilder lassen sich dann nach dem Speichern mit `<img src="your img url or path">` in einer Markdown-Zelle direkt in das Notebook einfügen.

> Hinweis:
> `.ig` Datenformat ist ähnlich wie ein `.zip` Archiv und enthält eine oder mehrere Ereignisdateien und wird von ISpy verwendet, das von CMS zur Anzeige von Ereignissen genutzt wird.

Mit einer Internetverbindung:

In [ ]:
%%html
<iframe src="https://ispy-webgl.web.cern.ch/ispy-webgl/" width="100%" height="700"></iframe>

Ohne eine Internetverbindung: Öffnen Sie die Datei `index.html` aus dem [Github Repository](https://github.com/cms-outreach/ispy-webgl) lokal in einem Webbrowser, nachdem Sie das GitHub Repository sich zuvor lokal kopiert haben.

Die wesentliche Aufgabe in diesem Abschnitt ist eine manuelle Klassifizierung der Ergebnisse nach ihren Zerfallskanälen anhand der Ereignisbilder. Die Klassifizierung wird später mit einer für diesen Zweck entwickelten Software automatisiert, um eine große Anzahl von Ereignissen in kurzer Zeit zu analysieren, anstatt jedes einzelne Kollisionsereignis Bild für Bild zu betrachten.


Der Fokus dieser Übung liegt auf der Wiederentdeckung des Zerfalls des Higgs-Bosons in zwei Z-Bosonen, die wiederum in vier geladene Leptonen zerfallen, $H\rightarrow ZZ\rightarrow 4\ell$.
Von allen geladenen Leptonen werden nur Elektronen und Myonen in der Analyse verwendet, da die Zerfälle des Z-Bosons in zwei $\tau$-Leptonen, $Z\rightarrow\tau\tau$, viel schwieriger zu handhaben sind,
und es schwieriger sein wird, das $H\rightarrow ZZ$-Signal von Hintergrundsignalen zu unterscheiden.


>Ausführliche Erklärungen:
>
>Die $\tau$-Leptonen zerfallen, bevor sie die erste Schicht des Spurendetektors erreichen, aber es ist möglich, sie aus den beobachteten Endzuständen zu rekonstruieren. Die Schwierigkeit ergibt sich aus den zusätzlichen Neutrinos in den $\tau$-Leptonenzerfällen. Da diese einen Teil der Energie wegtragen, wird der $H\rightarrow ZZ$-Peak in der ${m}_{4\ell}$-Verteilung - $\ell$ entspricht in diesem Fall den sichtbaren Zerfallsprodukten des $\tau$-Leptons -
verschmiert und zu niedrigeren Energien hin verschoben. Daher ist es viel schwieriger, $H\rightarrow ZZ$ Ereignisse von dem Untergrund zu unterscheiden (insbesondere $Z\rightarrow 4 \ell$),
wenn $\tau$-Leptonen verwendet werden.

### Mögliche Lösung:
-------------------------------------
* $\mathrm{H} \rightarrow \mathrm{ZZ} \rightarrow 4\ell$    
  (Bitte ergänzen Sie hier Ihre Notizen und fügen die ausgenommenen Bilder ein.)
* $\mathrm{H} \rightarrow \gamma \gamma$    
 (Bitte ergänzen Sie hier Ihre Notizen und fügen die ausgenommenen Bilder ein.)
* $\mathrm{H} \rightarrow \mathrm{W}^+ \mathrm{W}^- \rightarrow 2\ell 2\nu$    
  (Bitte ergänzen Sie hier Ihre Notizen und fügen die ausgenommenen Bilder ein.)
--------------------------------------

# Datenformat
-------------------------------

Im Verlauf dieser Übung werden benutzerdefinierte Klassen vorgestellt, die den Zeitaufwand und die Komplexität bei der Verarbeitung der Datensätze reduzieren. Diese Klassen bauen auf Paketen wie `numpy`, `pandas` oder `matplotlib` auf und fassen Schritte zusammen, die sonst bei der Verwendung dieser Pakete durchgeführt werden müssten, z. B. um ein Histogramm zu erstellen.

Im Allgemeinen handelt es sich um eine gängige Vorgehensweise, die Sie auch auf Ihre zukünftigen Analysen anwenden können: Es ist nicht notwendig, jedes Mal, wenn Sie neue Datensätze verarbeiten möchten, alles von Grund auf neu zu schreiben. Stattdessen ist es sinnvoll, eine Zwischenschicht zwischen Ihren Analyse-Workflows und den vorhandenen Paketen zu schaffen, die üblicherweise in einem Analyse-Software-Framework zusammengefasst sein können.

Die verschiedene Größen, die für die in dieser Übung durchgeführte Analyse benötigt werden, werden aus den einzelnen Ereignissen in mehrere Dateien geschrieben. In diesem Schritt wurde auch eine sehr grobe Vorauswahl der Ereignisse getroffen und nur die für die Analyse notwendigen Variablen herausgeschrieben: bestimmte Informationen über die einzelnen Leptonen. Die Original-Datensätze sind mehrere TB groß und eine zeiteffiziente Verarbeitung erfordert einen Cluster von Worker-Nodes, auf denen sie mehrere Stunden laufen muss. Die im Folgenden verwendeten Datensätze sind dagegen nur wenige MB groß und können im Rahmen dieser Übung recht schnell prozessiert werden.

Das in dieser Übung verwendete Datenformat, aus dem alle benötigten Größen entnommen werden, ist ein modifiziertes, von Menschen lesbares `.csv`-Format.

Ein Kollisionsereignis besteht aus Informationen zu verschiedenen Größen, die für jeden Ereignisdatensatz (eine Zeile) mit `;` getrennt werden. Jede Größe - wie z. B. **muon_px** - wird als Zeichenkette aus Float- oder Integer-Zahlen gespeichert, die durch `,` getrennt sind, um der Tatsache Rechnung zu tragen, dass es mehrere Leptonen in einem Ereignis enthalten sein können.

In [ ]:
%matplotlib inline
#%matplotlib qt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# Background simulation
name_1 = "../data/for_long_analysis/mc_init/MC_2012_ZZ_to_4L_2el2mu.csv"
name_2 = "../data/for_long_analysis/mc_init/MC_2012_ZZ_to_4L_4mu.csv"
name_3 = "../data/for_long_analysis/mc_init/MC_2012_ZZ_to_4L_4el.csv"

# Signal simulation
# name_1 = "../data/for_long_analysis/mc_init/MC_2012_H_to_ZZ_to_4L_2el2mu.csv"
# name_2 = "../data/for_long_analysis/mc_init/MC_2012_H_to_ZZ_to_4L_4mu.csv"
# name_3 = "../data/for_long_analysis/mc_init/MC_2012_H_to_ZZ_to_4L_4el.csv"

dataframe_1 = pd.read_csv(name_1, delimiter=";")
dataframe_2 = pd.read_csv(name_2, delimiter=";")

In [ ]:
dataframe_1

In [ ]:
dataframe_2

Angaben zu den in den Datensätzen enthaltenen Größen:

Mit einem Präfix (**Muon_** oder **Elektron_**) versehene Größen sind leptonenspezifisch. In Datensätzen, die den gemischten Kanal $2e2\mu$ enthalten, führt dieses Präfix zu einer klaren Unterscheidung zwischen den beiden Leptonentypen.

- **Run**, **Event** und **Luminosity_section** sind ereignisspezifische Variablen und stellen die Eindeutigkeit der einzelnen Ereignisse sicher
- **energie** und **charge** sind die Energien (in GeV) und die elektrischen Ladungen (in 1$e$-Einheiten) der Leptonen in einem Ereignis
- (**px**, **py**, **pz**) sind die jeweiligen Impulskomponenten (in GeV) der detektierten Leptonen
- **relPFIso** ist die relative Isolation der Leptonen. Sie ist die Summe über die Transversalimpulse aller nicht-leptonischen Teilchen geteilt durch den Transversalimpuls des betrachteten Leptons, berechnet in einem Kegel in $(\Delta\eta, \Delta\phi)$ um das Lepton. Um eine ungenaue Rekonstruktion und Fehlidentifikation mit anderen Teilchen zu vermeiden, ist eine gute relative Isolation wichtig: je kleiner der Wert, desto besser, was zu weniger Teilchen um das betrachtete Lepton herum führt.
- **SIP3d** ist die Signifikanz des 3D-Stoßparameters, und **dxy** und **dz** sind die Stoßparameter (in cm) quer bzw. längs in Bezug auf die Strahlachse. Diese Größen erlauben es, Leptonen aus primären Zerfällen, wie von einem Z-Boson, von Leptonen zu unterscheiden, die aus Zerfällen von langlebigen Teilchen, wie B-Mesonen, stammen, die außerhalb des primären Vertizes stattfinden.

Um auf Mengen einzelner Teilchen - in diesem Fall Myonen oder Elektronen - zuzugreifen, können die Strings (`str`) mit `,` in Listen (`list`) von Zahlen umgewandelt werden.
Dazu kann die Bibliothek `ast` verwendet werden, oder die vorhandene Methode `split` von `string`:

In [ ]:
import ast

px = ast.literal_eval(f"[{dataframe_2.loc[5, 'muon_px']}]")
px = np.array(px, dtype=float)
px

In [ ]:
py = dataframe_2.loc[5, 'muon_py'].split(",")
py = np.array(py, dtype=float)
py

In den beiden obigen Befehlen wurde auf den vierten Kollisionsereignis - Zählbeginn ist 0 - mit dem Index "5" zugegriffen, was der vierten Zeile in der `.csv`-Datei entspricht. Im ersten Befehl wurde mit der Python-Bibliothek `ast` eine Liste der x-Komponente des Impulses der Myonen in diesem Ereignis erzeugt. In diesem Falle sind fünf Myonen gespeichert. Mit dem zweiten Befehl wird eine Liste der y-Komponente der Myonenimpulse erstellt, diesmal mit der Methode `split` eines Strings. Auch hier besteht die Liste aus fünf Elementen, die den fünf Myonen im vierten Ereignis entsprechen.

Die Sortierung beider Listen ist gleich, so dass die x- und die y-Komponente an der entsprechenden Stelle in der Liste demselben Myon zugeordnet sind.

Basierend auf den Größen aus der .csv-Datei können neue Größen konstruiert werden. Im Falle des obigen Beispiels kann der Transversalimpuls `muon_pt` aus `muon_px` und `muon_py` konstruiert werden, wobei die Vorteile
der Bibliothek "numpy" nutzen, um Rechenoperationen mit dem gesamten Array durchzuführen:

In [ ]:
pt = np.sqrt(px ** 2 + py ** 2)
pt

Ein wesentlicher Schritt bei der Analyse von Kollisionsereignissen ist die Selektion von möglichst gut rekonstruierten und identifizierten Physikobjekten, in diesem Fall also Myonen und Elektronen, mit dem Ziel, das Signal anzureichern und den Untergrund zu reduzieren.

Einer der ersten Schritte bei einer solchen Auswahl kann an den Transversalimpulsen der Myonen demonstriert werden, die mit den oben genannten Befehlen konstruiert wurden. Es ist sinnvoll, Myonen mit einem
Transversalimpuls größer als 5 GeV zu verlangen, um schlecht rekonstruierte und falsch identifizierte Myonen zu entfernen.

In [ ]:
pt_minimum_filter = pt > 5
pt_minimum_filter

In [ ]:
pt = pt[pt_minimum_filter]
pt

Wie bisher wurde die besondere Stärke von `numpy` ausgenutzt: der Vergleich einer ganzen Liste. Im ersten Schritt wird eine Liste mit Booleschen Werten erzeugt, die angeben, ob die Bedingung `pt > 5` erfüllt wurde oder nicht. Danach kann diese verwendet werden, um Elemente, die die Bedingung nicht erfüllen, zu verwerfen, indem es als Index-Argument für die `pt`- Liste verwendet wird. Wie erwartet, wird der letzte Eintrag in der Liste entfernt.


Die boolesche Liste wirkt wie ein Filter auf die `pt` - Liste. Um die unerwünschten Myonen konsequent aus einem Ereignis zu verwerfen, muss der Filter auf **alle** Größen angewendet werden, die sich auf Myonen in dem betrachteten Ereignis beziehen. Dieser Filterschritt muss für jedes Kollisionsereignis, das in den Datensätzen gespeichert ist, wiederholt werden, und die bearbeiteten Datensätze müssen erneut gespeichert werden.

Um den Aufwand dieser zeitintensiven Aufgabe deutlich zu reduzieren, wurde im Rahmen dieser Übung eine eigene, dafür geeignete Klasse erstellt,
die die Filterschritte und das Anlegen neuer Mengen automatisch durchführt.

# Anwendung von Filtern mit der benutzerdefinierten Klasse Apply
-----------------------------------

Während einer Analyse ist es in der Regel erforderlich, neue Mengen zu erstellen, physikalische Objekte (in unserem Fall Myonen und Elektronen) auszuwählen, die die Selektionskriterien erfüllen. Auf diese Weise werden nur die für die Analyse interessanten Daten und Informationen ausgewählt, was den Umfang der zu analysierenden Informationen reduziert.

Diese Aufgaben wurden in dem kleinen Beispiel im vorherigen Abschnitt demonstriert:

* Wir haben den Transversalimpuls der Myonen `pt` in einem Kollisionsereignis eingeführt, indem wir ihn aus `px` und `py` der Myonen berechnet haben,
* wir stellten fest, ob die Myonen die Anforderung `pt > 5` erfüllen, was zur Auswahl von 4 aus 5 Myonen führte. Dies müsste auf alle Größen des Kollisionsereignisses angewendet werden.
* Zusätzlich kann gefordert werden, dass ein Kollisionsereignis mindestens 4 Myonen mit `pt > 5` haben sollte, andernfalls müsste ein solches Ereignis verworfen werden, da keine Rekonstruktion eines Z-Bosonen-Paares möglich ist.

Um diese Aufgaben zu erfüllen, wird eine eigene Klasse `Apply` verwendet, die in diesem Abschnitt vorgestellt wird und ein Beispiel für ein rudimentäres Analyse-Framework darstellt. Mit ihr können verschiedene Selektions-, Filter- und Rekonstruktionsschritte durchgeführt, die Zwischenergebnisse visualisiert oder die entstandenen reduzierten Datensätze gespeichert werden.

Im Rahmen dieser Übung werden im Folgenden Codevorlagen gegeben, die Sie in den nächsten Abschnitten vervollständigen sollen, um in den einzelnen Analyseschritten voranzukommen.

Als erstes wird die Klasse importiert:

In [ ]:
from include.processing.Apply import Apply

Beim Erzeugen eines `Apply`-Objekts müssen eine Klasse die die Filter enthält (Filterklasse) und eine Klasse, die die notwendigen Berechnungen durchführt (Kalkulationsklasse) an `Apply` übergeben werden, damit es richtig funktioniert. Dies geschieht mit den später verwendeten Keyword-Argumenten `calc_instance` und `filter_instance`. Diese beiden übergebenen Klassen bilden eine Momentaufnahme der zum Zeitpunkt der Initialisierung vorhandenen Funktionen, die `Apply` in den Filter- und Rekonstruktionsschritten verwendet. Ein Beispiel ist im Folgenden dargestellt.

Im Detail müssen die beiden Klassen die Berechnung für neue Variablen (`calc_instance`) und die Auswahlkriterien der Filter (`filter_instance`) enthalten. Die Methoden innerhalb dieser Klassen sollten als eigenständige mit `@staticmethod` erstellt und logisch in zwei Klassen zusammengefasst werden.

Das Beispiel aus dem vorherigen Abschnitt kann für die `Apply`-Klasse wie folgt realisiert werden:

In [ ]:
class Calc_Start(object):
    
    @staticmethod
    def pt(px, py):
        return np.sqrt(px ** 2 + py ** 2)

class Filter_Start(object):
    
    @staticmethod
    def pt_min(pt, look_for):
        return pt > 7.0 if look_for == "electron" else (pt > 5.0 if look_for == "muon" else None)

Die implementierten Python-Funktionen `pt` und `pt_min` innerhalb der Klassen `Calc_Start` und `Filter_Start` können nun wie folgt an die Klasse Apply übergeben werden:

In [ ]:
process = Apply(file=name_2, nrows=2000,
                calc_instance=Calc_Start, filter_instance=Filter_Start)

Bei `file` muss der Pfad zum jeweiligen Datensatz angegeben werden, `nrows` gibt an, wie viele Zeilen, d.h. Ereignisse, gelesen werden - wenn kein `nrows` angegeben wird (entspricht `nrows=None`), werden alle Zeilen, d.h. alle Ereignisse, eingelesen. Für das Beispiel ist es ausreichend, eine kleine Anzahl von Ereignissen zu betrachten. Für Testzwecke Ihrer Implementationen von Filtern und Berechnung von neuen Größen ist es in der Regel ausreichend, auf einer Teilmenge von Ereignissen zu arbeiten, also denken Sie bitte daran, wenn Sie Ihre Analyse implementieren.

Sobald Sie sich auf Ihre Auswahl und Größen festgelegt haben, sollten Sie die Analyse natürlich für alle Ereignisse durchführen - mehr dazu später.

Die so implementierten Funktionen (`pt` in `Calc_Start` und `pt_min` in `Filter_Start` im oben angelegten `Apply`-Objekt `process`) reichen allein nicht aus, um die gewünschte Analyse durchzuführen, da, wie im vorigen Abschnitt gezeigt wurde, die Prozedur nur auf eine Menge angewendet wird und nicht auf alle, die im Ereignis enthalten sind.

Daher werden diese Funktionen in bereits implementierte *Workflows* der Klasse `Apply` eingebunden, die die gewünschte Anwendung der angegebenen Funktionen auf den gesamten Ereignisdatensatz durchführen.

Eine Übersicht über alle vorhandenen Workflows kann wie folgt aufgerufen werden:

In [ ]:
Apply.help()

Eine Zusammenfassung dessen, was in einem bestimmten Workflow ausgeführt wird, kann wie folgt nachgesehen werden:

In [ ]:
Apply.help("filter", "pt_min")

oder:

In [ ]:
Apply.help("filter", "pt_exact")

Im obigen Beispiel sehen Sie, dass sich die beiden Workflows zwar unterscheiden, aber ähnliche Funktionssätze verwenden:

* Der Workflow `"pt_min"` entfernt die Leptonen aus dem Ereignis, die einen Transversalimpuls unterhalb eines festgelegten Schwellenwertes haben. Wenn weniger als 4 Leptonen übrig sind (4 Elektronen, 4 Myonen bzw. 2 Elektronen und 2 Myonen), wird das Ereignis verworfen.
* Im Gegensatz dazu verwirft der Workflow `"pt_exact"` das gesamte Ereignis, wenn die einzelnen Leptonen die exakten Kriterien nicht erfüllen.

Zusammenfassend demonstriert dieses Beispiel den Unterschied zwischen einem leptonenbasierten Filter, der den Inhalt des Ereignisses (die Leptonenanzahl) verändern kann, und einem ereignisbasierten Filter, der nur zum Verwerfen von Ereignissen verwendet wird und den Ereignissatz in keiner Weise verändert.

Der Aufruf von `Apply.help(workflow, method)` zeigt die Workflows, sowie die dort benötigten Funktionen. Da wir aber bereits ein `Apply`-Objekt `process` initialisiert und ihm bestimmte `calc_instance` (`Calc_Start`) und `filter_instance` (`Filter_Start`) übergeben haben, ist es möglich zu prüfen, ob alle notwendigen Methoden für einen Workflow auch im `process`-Objekt vorhanden sind:

In [ ]:
process.help("filter", "pt_min")

In [ ]:
process.help("filter", "pt_exact")

Im Workflow `"pt_min"` werden z.B. nur die Methoden `Calc_Start.pt` und `Filter_Start.pt_min` verwendet, wobei `Filter_Start.pt_min` als Filter wirkt. Für diesen Workflow sind alle notwendigen Teile implementiert.

Im Gegensatz dazu benötigt der Workflow `"pt_exact"` noch die Methode (den Filter) `FilterStudent.pt_exact`, die in den nächsten Abschnitten von Ihnen implementiert wird.


Es ist also möglich, sofort den `"pt_min"` Workflow zu verwenden. Dort wird der Transversalimpuls `pt` implizit als Methode während des Workflows hinzugefügt.

Zu Demonstrationszwecken wollen wir vorher den Transversalimpuls explizit hinzufügen, der in dem mit `name_2` aufgerufenen Datensatz noch nicht vorhanden ist. In einem Zwischenschritt, der bereits im Workflow `"pt_min"` enthalten ist, wird der Transversalimpuls `"pt"` zum Datensatz `name_2` hinzugefügt, indem die unten angegebenen benötigten Schritte ausgeführt werden:

In [ ]:
dataframe_2

In [ ]:
process.add("pt")
process.data

Die Darstellung der Verteilung einer solchen Variablen kann mit der Methode `hist(variable, bins, hist_range)` erfolgen.

In [ ]:
process.hist(variable="pt", bins=50, hist_range=(0, 80))

Obwohl diese Methode auch `hist` genannt wird, unterscheidet sie sich von dem bereits bekannten `plt.hist` aus `matplotlib.pyplot`. Unter der Haube wird auch `plt.hist` verwendet, aber zusätzlich wird für die korrekte Zuweisung und Konvertierung von Mengen, die als Strings im Datensatz gespeichert sind, geachtet.

Die Anwendung des Filters für den minimalen Transversalimpuls und das Entfernen aller Ereignisse, die weniger als vier Leptonen enthalten, ergibt die entsprechende Verteilung:

In [ ]:
process.filter("pt_min")

In [ ]:
import matplotlib.pyplot as plt

def pt_hist_helper(title, lepton_number=None):
    process.hist(variable="pt", bins=50, hist_range=(0, 80), lepton_number=lepton_number)
    # Sie können auf die aktuelle 'axis' mit plt.gca (GetCurrentAxis) oder die aktuelle 'figure' mit plt.gcf zugreifen.
    # von hier an ist es Ihnen überlassen, den Plot so anzupassen, wie Sie es möchten
    ax = plt.gca()
    ax.set_title(title)
    ax.set_ylabel("N")
    ax.set_xlabel("$p_T$ in GeV")
    # plt.savefig(f"histogramms/background_4mu_{title.replace('$', '').replace(' ', '_')}.png")
    plt.show()

pt_hist_helper("$p_T$ of all leptons")
pt_hist_helper("$p_T$ of first lepton", 0)
pt_hist_helper("$p_T$ of third and fourth lepton", [2, 3])
pt_hist_helper("$p_T$ of fourth and fifth lepton", [3, 4])

<div class="alert alert-info">
Andere Variablen aus dem/den Datensatz/en können ebenfalls im Histogramm mit der Methode 'hist' auf ähnliche Weise visualisiert werden. Bitte erweitern Sie den obigen Code, um die Auswirkung des Filterschritts mit der Anforderung auf den Myonen-Transversalimpuls zu sehen, indem Sie andere im Ereignissatz vorhandene Variablen vor und nach dem Filterschritt betrachten. Bei welchen Variablen gibt es signifikante Änderungen durch die Filterbedingung?

Sehen alle diese Verteilungen so aus, wie Sie es erwarten würden? Falls nicht, woran könnte es liegen
?
</div>

In [ ]:
# Bitte hier Ihren Code einfügen.
# Visualisieren Sie weitere Größen im Datensatz hier (vor/nach der Anforderung).
# Sie können auch hierher zurückkommen, um Codefragmente zur Visualisierung Ihrer weiteren implementierten Anforderungen zu verwenden.

# Berechnung von wichtigen Variablen
--------------------------------

In diesem Abschnitt werden nun weitere für die Analyse notwendige Variablen implementiert, ähnlich wie dies für den Transversalimpuls geschehen ist. Die Details zur Implementierung der jeweiligen Funktionen sind in den unten angegebenen Dokumentationen der Funktionsvorlagen kurz zusammengefasst. Es werden zusätzliche Informationen zu den Eingabeargumenten und deren Struktur gegeben, und wie die Ausgabe aussehen sollte, um mit den jeweiligen Workflows der einzelnen Schritte der `Apply`-Klasse kompatibel zu sein. In diesem Abschnitt der Übung sind drei neue Größen zu berechnen.

<div class="alert alert-info">

Implementieren Sie am Beispiel des Transversalimpulses die folgenden nachträglich notwendigen Größen:
  * Pseudorapidität $\eta$
  * Azimutwinkel $\phi$
  * Das Quadrat der invarianten Masse $m_{\mathrm{inv}}$

Warum reicht es aus, die Pseudorapidität anstelle der Rapidität zu betrachten? Was ist der Unterschied zwischen den beiden Größen? Stellen Sie auch die Verteilungen der Pseudorapidität und des Azimutwinkels dar. Gibt es Auffälligkeiten in den Verteilungen? Erläutern Sie Ihre Beobachtungen.

> _Hinweis_: Sie können Ihre Visualisierungen am Beispiel des Transversalimpulses orientieren. Wenn Sie ein neues `Apply`-Objekt erzeugen, achten Sie darauf, dass Sie immer die richtige `calc_instance` und `filter_instance` übergeben. Sie können diese Aufgabe auch mit einer verringerten Anzahl von Ereignissen durchführen, wie im Beispiel des vorherigen Abschnitts gezeigt (`nrows=2000` bei der Erstellung des `Apply`-Objekts).
</div>

Die zu erstellende Klasse erbt von der bereits bekannten Klasse, die den Transversalimpuls enthält. Zusätzlich wird später noch eine Klasse hinzugefügt, die weitere bereits implementierte Methoden enthält. Letztendlich interagieren viele der bereits implementierten Methoden und die von Ihnen implementierten Methoden miteinander, daher ist es wichtig, dass die __Methodennamen__, sowie die __Namen und die Anzahl der Argumente unverändert bleiben__.

Im Weiteren werden die Erklärungen der einzelnen Funktionen übersetzt. Die skizzierte Implementation, sowie weitere Informationen über die Variablen werden dagegen typisch in englischer Sprache aufgeführt. Sie werden - falls sie es nicht bereits getan haben - vieles an Code und seiner Dokumentation zumeist nur in englischer Sprache antreffen.

In [ ]:
class CalcStudent(Calc_Start):
    '''
    Klasse für die Berechnung von bestimmten Größen, die für
    die Selektionsanforderungen oder für die Rekonstruktion notwendig sind.
    '''
    
    @staticmethod
    def phi(px, py):
        """
        Diese Funktion berechnet den Azimutwinkel phi für 
        jedes Lepton im Ereignis. Die gleichen Listeneinträge von 
        der Argumente entsprechen den Größen gleicher 
        Leptonen.
        
        ------------------------------------------------------
        
        :param px: ndarray
                   1D array containing data with "float" type.
        :param py:  ndarray
                    1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
                 
        Exemplary: 
            number of leptons in the event: 5
            
            :param px: np.array([px_1, px_2, px_3, px_4, px_5])
            :param py: np.array([py_1, py_2, py_3, py_4, py_5])
            
            :return: np.array([phi_1, phi_2, phi_3, phi_4, phi_5])
        """
        # Bitte vervollständigen Sie die Methode mit Ihrem Code unter Berücksichtigung 
        # der Ein- und Ausgabespezifikationen aus der Dokumentation
    
    @staticmethod
    def pseudorapidity(px=None, py=None, pz=None, energy=None):
        """
        Diese Funktion berechnet die Pseudorapidität für jedes Lepton in 
        einem Ereignis. Der Parameter 'energy' ist optional und kann, 
        falls nicht notwendig, bei der Berechnung vernachlässigt werden, 
        sollte aber in der der Argumentenliste wie oben gezeigt angegeben 
        werden. Die gleichen Listeneinträge der Argumente entsprechen den 
        Größen gleicher Leptonen.
        
        ------------------------------------------------------

        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                    1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        
        Exemplary: 
            number of leptons in the event: 5
            
            :param px: np.array([px_1, px_2, px_3, px_4, px_5])
            :param py: np.array([py_1, py_2, py_3, py_4, py_5])
            :param pz: np.array([pz_1, pz_2, pz_3, pz_4, pz_5])
            :param energy: np.array([e_1, e_2, e_3, e_4, e_5]) (optional)
            
            :return: np.array([psdrapid_1, psdrapid_2, psdrapid_3, psdrapid_4, psdrapid_5])
        """
        # Bitte vervollständigen Sie die Methode mit Ihrem Code unter Berücksichtigung 
        # der Ein- und Ausgabespezifikationen aus der Dokumentation
        
    @staticmethod
    def invariant_mass_square(px, py, pz, energy=None):
        """
        Diese Funktion berechnet das Quadrat (!) der invarianten Masse. Die 
        Größe der Eingabefelder hängt davon ab, wie viele Leptonen in dem 
        Ereignis vorhanden sind. Der Parameter 'energy' ist optional und kann, 
        falls nicht notwendig, bei der Berechnung vernachlässigt werden, 
        sollte aber in der Argumentenliste wie oben gezeigt beibehalten werden. 
        Die gleichen Listeneinträge der Argumente entsprechen den Größen 
        gleicher Leptonen.
        
        ------------------------------------------------------
        
        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                   1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: "float"
        
        Exemplary: 
            number of leptons: 4 energy is not None
            
            :param px: np.array([px_1, px_2, px_3, px_4])
            :param py: np.array([py_1, py_2, py_3, py_4])
            :param pz: np.array([pz_1, pz_2, pz_3, pz_4])
            :param energy: np.array([e_1, e_2, e_3, e_4])
            
            :return: float(mass_square)
            
            or:
            number of leptons: 2 energy is None
            
            :param px: np.array([px_1, px_2])
            :param py: np.array([py_1, py_2])
            :param pz: np.array([pz_1, pz_2])
            :param energy: np.array([e_1, e_2])
            
            :return: float(mass_square)
            
        """
        # Bitte vervollständigen Sie die Methode mit Ihrem Code unter Berücksichtigung 
        # der Ein- und Ausgabespezifikationen aus der Dokumentation

# Erstellung der Selektionsbedingungen
------------------------------

Wie am Beispiel der Bedingung des minimal notwendigen Transversalimpulses gezeigt, werden in diesem Abschnitt weitere Selektionsbedingungen implementiert und von Ihnen definiert. Die Grenzwerte für die einzelnen Bedingungen können Sie selbst wählen. Dazu ist die Visualisierung der Verteilungen, wie Sie sie bereits in den vorherigen Abschnitten gesehen haben, hilfreich.

Probieren Sie ruhig verschiedene Einstellungen für die Selektionsbedingungen aus, da die Signalempfindlichkeit Ihrer Analyse je nach Wahl eines Schwellenwertes für eine Größe variieren kann. Versuchen Sie daher, eine sinnvolle Kombination zu finden.

Eine Begründung für die von der CMS-Kollaboration gewählten Schwellenwerte für die Entdeckung des Higgs-Bosons im Jahr 2012 finden Sie in der [**offiziellen Veröffentlichung**](https://arxiv.org/pdf/1207.7235.pdf). Dieses Paper ist eine Kombination aller Zerfallskanäle des Higgs-Bosons, die für die Entdeckung untersucht und kombiniert wurden, daher können Sie sich beim Lesen auf den Zerfall in vier Leptonen konzentrieren. Falls Sie Interesse haben, können Sie sich auch die anderen Zerfallskanäle anschauen - für die Übung ist das aber nicht notwendig.

Sie können die Wahl der Selektionsschwellen auch in Bezug auf die im Paper vorgeschlagenen Werte variieren, um zu versuchen, ein besseres Ergebnis zu erhalten. Denken Sie aber daran, dass die Wahl der Werte für die Schwellenwerte auf Untersuchungen mit den MC-Simulationen und nicht auf den gemessenen Daten beruhen sollte, um eine voreingenommene Suche nach einem Signalpeak zu vermeiden.

<div class="alert alert-info">

* Vervollständigen Sie die Methoden in der Klasse `FilterStudent` und wählen Sie die passenden Schwellenwerte aus (siehe auch [**offizielle Veröffentlichung**](https://arxiv.org/pdf/1207.7235.pdf)).  Visualisieren Sie die Anwendung dieser Selektionsfilter exemplarisch - werden auch andere Variablen durch die Anwendung der jeweiligen Filter beeinflusst?
* Es wird auch eine Bedingung an die relative Isolation gestellt. Warum ist es wichtig, diese näher zu betrachten?
* Für den Pseudorapiditätsfilter ist es notwendig, eine Unterscheidung zwischen Elektronen und Myonen zu treffen. Erläutern Sie dies. (_Hinweis_ : Betrachten Sie dieses [**Bild**](http://hep.fi.infn.it/CMS/software/ResultsWebPage/Images/Geometry/Tracker_SubDetectors_x_vs_eta.gif))
* Weiterhin soll ein Filter für die rekonstruierten Z-Massen gemacht werden. Warum ist das notwendig?

    
> _Hinweis 1_: Sie können sich bei der Visualisierung an dem Beispiel des Transversalimpulses orientieren. Wenn Sie ein neues `Apply`-Objekt erzeugen, achten Sie darauf, dass Sie immer die richtige `calc_instance` und `filter_instance` übergeben. Sie können diese Aufgabe wieder für eine kleinere Anzahl von Ereignissen durchführen, wie im Beispiel zuvor gezeigt (`nrows=2000` in der `Apply`-Objekt-Erstellung).
>
> _Hinweis 2_: Nachdem Sie die Klasse `FilterStudent` fertiggestellt haben, führen Sie die Zelle mit dem Kommentar `# Technische Notwendigkeit` nach der Implementierung der Klasse `FilterStudent` aus. Danach können Sie für Ihre Testzwecke ein `Apply`-Objekt mit einem vollständigen Satz von Methoden erzeugen.
</div>

In [ ]:
class FilterStudent(Filter_Start):
    '''
    Class that introduces requirements with certain thresholds chosen by user 
    to restrict the leptons in the events.
    '''
    
    @staticmethod
    def combined_charge(charge, combination_number):
        """
        Es wird überprüft, ob eine elektrisch neutrale Ladungskombination möglich 
        ist. Mit 'combination_number = 4' sollen alle vierfachen Kombinationen 
        der Elemente der elektrischen Ladung gebildet werden - auch wenn es mehr 
        als vier Leptonen im Ereignis sind. Mit 'combination_number = 2' werden 
        alle zweiartige Kombinationen der Elemente der elektrischen Ladung 
        gebildet werden - auch hier: es könnten mehr als zwei Leptonen sein, die 
        übergeben werden. Die Größe der Liste der Ladungen hängt davon ab, wie 
        viele Leptonen in dem Ereignis vorhanden sind.
        
        Hinweis: Es werden entweder Elektronen oder Myonen an diese Methode 
        übergeben, niemals eine gemischte Sammlung von Leptonen. Eine 
        Unterscheidung zwischen "electron" und "muon" ist daher in der 
        Implementierung nicht notwendig.
        
        ------------------------------------------------------
        
        :param charge: ndarray
                       1D array containing data with "int" type.
        :param combination_number: int
                            4 if lepton_type is not "both", 2 else
        :return: bool
        
        Exemplary: 
            combination_number = 4; 
            :param charge: np.array([-1, 1, 1, -1, -1])            
            -> possible four-like combinations are 0 and -2
            -> a neutral electrical combination of four leptons is possible
            :return: True 
            
            or:
            combination_number = 2; 
            :param charge: np.array([-1, -1, -1, -1]) 
            -> possible two-like combination is -2
            -> a neutral electrical combination of two leptons is not possible
            :return: False
        """
        # Bitte vervollständigen Sie die Methode mit Ihrem Code unter Berücksichtigung 
        # der Ein- und Ausgabespezifikationen aus der Dokumentation
        
    @staticmethod
    def pt_exact(p_t, lepton_type=None):
        """
        Diese Funktion prüft, ob die genaue Anforderung an den Transversalimpuls 
        erfüllt ist. Die Werte, die von der CMS-Gruppe gewählt wurden, sind: 
            (pt>20 GeV: >= 1; pt>10 GeV: >= 2; pt>pt_min: >= 4) 
        
        Hinweis: Für diese Methode werden gemischte Leptonensammlungen 
        bereitgestellt, wenn lepton_type='both' (im 2el2mu-Kanal) gilt. Eine 
        Unterscheidung zwischen "electron" oder "muon" ist nicht notwendig, wenn 
        zuvor der Workflow pt_min ausgeführt wurde. Wenn Sie erneut prüfen 
        wollen, ob das Kriterium für den minimalen Transversalimpuls erfüllt ist, 
        müssen Sie berücksichtigen, ob lepton_type='electron' oder 
        lepton_type='muon' ist.
        
        ------------------------------------------------------
        
        :param p_t: if lepton_type != "both" 
                        ndarray 
                        1D array containing data with `float` type.
                    if lepton_type == "both"
                        (ndarray, ndarray): (pt_muon, pt_electron)
                        two 1D array containing data with `float` type.
        :param lepton_type: str
                            "muon" (4mu channel) or 
                            "electron" (4el channel) 
                            or "both" (2el2mu channel)
                            
        :return: boolean
                 True or False
        
        Exemplary: 
            :lepton_type: "electron" or "muon" or None
            :param p_t: np.array([22.3, 17.5, 9.4, 6.5])            
            -> the first lepton has a transverse momentum greater than 20 GeV.
            -> the second lepton has a transverse momentum greater than 10 GeV
            a) the workflow pt_min was already carried out so no further 
               steps are necessary 
               -> return True
            b) the workflow pt_min was not carried out before:
               -> if lepton_type == "muon" : 6.5 GeV are greater than the
                  minimum transverse momentum of muons (5 GeV)
               :return: True
               or
               -> if lepton_type == "electron": the last lepton do not pass 
                  minimum transverse momentum threshold of electrons (7 GeV)
               :return: False
            
            lepton_type: "both"
            :param p_t: (np.array([29.4, 16.5]), np.array(10.2, 7.5))
            -> pt_mu, pt_el = p_t
            -> Check the minimal transverse momentum if the workflow pt_min was 
               not carried out before for both transverse momentum arrays 
               (both satisfy the minimum requirement of transverse momentum)
            -> there is one lepton with pt > 20GeV and more than two leptons 
               with pt > 10 GeV
            :return: True
        """
        
        # Bitte vervollständigen Sie die Methode mit Ihrem Code unter Berücksichtigung 
        # der Ein- und Ausgabespezifikationen aus der Dokumentation
        
        if lepton_type == "both":
            pt_mu, pt_el = p_t
            # Fortsetzung der Verarbeitung
        if lepton_type != "both":
            p_t = p_t
            # Fortsetzung der Verarbeitung
    
    @staticmethod
    def pseudorapidity(eta, lepton_type):
        """
        Diese Funktion prüft, ob die Bedingung für Pseudorapidität 
        von den Leptonen im Ereignis erfüllt wird.
        
        
        Hinweis: Es werden entweder Elektronen oder Myonen an diese Methode 
        übergeben, niemals eine gemischte Sammlung von Leptonen. Nur eine 
        Unterscheidung zwischen "electron" und "muon" ist notwendig. Der 
        gemischte Zerfallskanal wird zweigeteilt und diese Methode wird 
        zweimal in dem entsprechenden Workflow aufgerufen.
        
        ------------------------------------------------------
        
        :param eta: ndarray
                    1D array containing data with "float" type.
        :param lepton_type: str
                            "muon" or "electron"
        :return: ndarray
                 1D array containing data with "bool" type.
        
        Exemplary: 
            :param lepton_type: "electron"; 
            :param eta: np.array([1.9, 1.8, -1.7, 1.9, -2.0])            
            -> each lepton satisfies the electron requirement of 
               (abs(eta) < 1.479 | abs(eta) > 1.653) & abs(eta) < 2.5
                   Note: In the range [1,479, 1,653] there are detector 
                         boundaries, which means that no exact reconstruction 
                         for electrons is possible there. Muons do not have 
                         this limitation.
            :return: True
        """
        
        # Bitte vervollständigen Sie die Methode mit Ihrem Code unter Berücksichtigung 
        # der Ein- und Ausgabespezifikationen aus der Dokumentation
        
        if lepton_type == "muon":
            # Fortsetzung der Verarbeitung
            pass
        if lepton_type == "electron":
            # Fortsetzung der Verarbeitung
            pass
    
    @staticmethod
    def impact_parameter(sip3d, dxy, dz):
        """
        Diese Funktion testet, ob die Variablen des Stoßparameters 
        bestimmte minimale Schwellenwerte erfüllen
        
        Hinweis: Es wird nicht zwischen Leptonentypen unterschieden.
        
        ------------------------------------------------------
        
        :param sip3d: ndarray
                      1D array containing data with "float" type.
        :param dxy: ndarray
                    1D array containing data with "float" type.
        :param dz: ndarray
                   1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "bool" type.
                 
        Exemplary:
            :param sip3d: np.array([sip3d_1, sip3d_2, sip3d_3, sip3d_4, sip3d_5])
            :param dxy: np.array([dxy_1, dxy_2, dxy_3, dxy_4, dxy_5])
            :param dz: np.array([dz_1, dz_2, dz_3, dz_4, dz_5])
            -> Check if spi3d dxy and dz satisfy the corresponding threshold value
            :return: np.array([bool, bool, bool, bool, bool])
        """
        # Bitte vervollständigen Sie die Methode mit Ihrem Code unter Berücksichtigung 
        # der Ein- und Ausgabespezifikationen aus der Dokumentation
    
    @staticmethod
    def relative_isolation(rel_iso_array):
        """
        Prüft, ob die relative Isolation den Wert überschreitet, der als
        Schwellenwert für die relative Isolation gewählt wurde oder nicht.
        
        Hinweis: Es wird nicht zwischen Leptonentypen unterschieden.
        
        ------------------------------------------------------

        :param rel_iso_array: ndarray
                              1D array containing data with `float` type.
        :return: ndarray
                 1D array containing data with `bool` type.
                 
        Exemplary:
            :param rel_iso_array: np.array([rel_iso_1, rel_iso_2, rel_iso_3, rel_iso_4])
            -> rel_iso_array < threshold value?
            :return: np.array([bool, bool, bool, bool])
        """
        # Bitte vervollständigen Sie die Methode mit Ihrem Code unter Berücksichtigung 
        # der Ein- und Ausgabespezifikationen aus der Dokumentation
        
    @staticmethod
    def zz(z1, z2):
        """
        Diese Funktion prüft, ob die beiden Z-Bosonen-Massen innerhalb dem 
        erlaubten Bereich liegen. Das Ziel ist eine Off-Shell-Rekonstruktion. 
        Aus diesem Grund muss z1 real und z2 virtuell sein. Außerdem sollte 
        gewährleistet werden, dass die Massen für z1 und z2 gilt: z1 > z2.
        
        ------------------------------------------------------

        :param z1: float
                   first Z-Boson mass
        :param z2: float
                   second Z-Boson mass
        :return: bool
                 True or False
        
        Exemplary: 
            :param z1: 91
            :param z2: 45
            -> It satisfy the requirement that z1 > z2
            -> if it satisfy the treshold values for z1 and z2
                :return: True
               else
                :return: False
        """
        # Bitte vervollständigen Sie die Methode mit Ihrem Code unter Berücksichtigung 
        # der Ein- und Ausgabespezifikationen aus der Dokumentation

Der folgende Code kombiniert die Vererbung bestehender und von Ihnen implementierter Methoden so, dass alle Methoden innerhalb der Workflows interoperabel sind und in der Apply-Klasse verwendet werden können. Nach dem Ausführen der Zelle kann beim Erzeugen eines `Apply`-Objekts die `filter_instance=Filter` und die `calc_instance=Calc` übergeben werden. Wenn der Code in den jeweiligen Methoden geändert wird, muss diese Zelle **immer zuerst** ausgeführt werden, um die Methoden in `Calc` und `Filter` zu aktualisieren.
> Diese Art der " Fragmentierung" der Klassen `Calc` und `Filter`, die an `Apply` übergeben werden, sollte in einem in Zukunft von Ihnen selbst erstellten Framework vermieden werden (z.B.: eine Klasse `Filter` und `Calc` mit allen notwendigen Methoden ist ausreichend). Die Entscheidung dieser "Fragmentierung" wurde ausschließlich im Zusammenhang mit dieser Aufgabe getroffen.

In [ ]:
# Technische Notwendigkeit
from include.processing.CalcAndAllowerInit import FilterInit
from include.processing.CalcAndAllowerInit import CalcInit

FilterInit.a_filter_instance = FilterStudent
FilterInit.a_calc_instance = CalcStudent


class Filter(FilterStudent, FilterInit):
    pass


CalcInit.c_filter_instance = Filter
CalcInit.c_calc_instance = CalcStudent


class Calc(CalcStudent, CalcInit):
    pass

# Anwendung von Filter- und Rekonstruktionsschritten auf simulierte und gemessene Datensätze
---------------------------------------

In diesem Abschnitt werden die zuvor von Ihnen implementierten Bedingungen an die Selektion und neuen Größen für die gesamten Datensätze angewendet und berechnet. Auch die tatsächliche Messung wird hier nun eingeführt. Wenn Sie also mit einigen Schwellenwerten der Anforderungen noch nicht zufrieden sind, sollten Sie diese ändern, bevor Sie diesen Abschnitt ausführen, da dieser Teil der Analyse einige Zeit in Anspruch nimmt.

Außerdem wird dadurch sichergestellt, dass Sie die Messung nicht vorher sehen und versuchen, Ihre Schwellenwerte der Anforderungen an die Messung anzupassen - dies würde der Idee einer blinden Analyse widersprechen, die vor dem Betrachten der Daten optimiert wird, um Voreingenommenheit durch die Subjektivität zu vermeiden.

> Wenn Sie dies auf einer Maschine ausführen, die mehrere Kerne zur Verfügung hat, können Sie dies durch das Argument "multi_cpu=True" und die Angabe "n_cpu=3" bei der Erstellung des Apply-Objekts angeben (oder eine beliebige andere Anzahl von CPU-Kernen, die Sie zur Verfügung stellen wollen). 

Für die unten gezeigte Routine kann eine Hilfsfunktion erstellt werden, die alle Datensätze in den Ordnern auflistet:

In [1]:
import os


def collect_all_dataset_paths(directories=None):
    """
    Dies ist eine Hilfsfunktion zum Sammeln aller Datensätze in angegebenen 
    Verzeichnissen.
    
    ------------------------------------------------------
    
    :param directories: str or list
                        path of a directory or list of paths to directories
    :return: list
             list of str, containing the dataset paths
    """
    
    if isinstance(directories, str):
        return [os.path.join(directories, file) for file in os.listdir(directories) if ".csv" in file]
    if isinstance(directories, list):
        _tmp = []
        for directory in directories:
            _tmp += collect_all_dataset_paths(directory)
        return _tmp

In [ ]:
files = collect_all_dataset_paths(["../data/for_long_analysis/mc_init/", "../data/for_long_analysis/ru_init/"])
files

Die Routine, die alle in Apply.help() aufgeführten Schritte in einer möglichen sinnvollen Reihenfolge abarbeitet, könnte wie folgt aussehen:
<div class="alert alert-info">

 * Warum ist es notwendig, den Filter auf die elektrische Ladung mehrmals durchzuführen?
 * (optional) Ist die Reihenfolge der angewandten Filter sinnvoll? Wenn nicht, dann können Sie sie entsprechend Ihrer vorstellunt ändern.
</div>

In [ ]:
def filter_and_reco_process(file):
    
    # Bereits bekannter Ladevorgang der einzelnen Datensätze
    process = Apply(file=file, calc_instance=Calc, filter_instance=Filter, multi_cpu=False)
    
    # Name of the record, without the name of the directory in which it is contained
    filename = process.filename
    
    # Name des Datensatzes, ohne den Namen des Verzeichnisses, in dem er enthalten ist
    directory = "../data/for_long_analysis/ru" if "CMS_Run" in filename else "../data/for_long_analysis/mc"
    
    # Die Anwendung der verschiedenen Bedingungen
    process.filter("electric_charge")
    
    # Wenn Sie die verarbeiteten Datensätze nach jedem Filterschritt speichern möchten, 
    # können Sie dies wie folgt tun:
    # process.save(os.path.join(f"{directory}_AfterFirstElectricCharge", filename))
    
    process.filter("relative_isolation")
    process.filter("impact_parameter")
    process.filter("pt_min")
    process.filter("pt_exact")
    process.filter("pseudorapidity")
    
    # Die folgenden beiden Filter führen eine schnelle Überprüfung durch, ob zwei oder vier Leptonenmassen in einem bestimmten Bereich existieren
    # Dieser Filter macht keine vollständige Rekonstruktion, bei der alle notwendigen Variablen und die weiteren 
    # Einschränkungen berücksichtigt werden. Sie dienen nur zur kurzen Abschätzung der Werte, ob eine 
    # Masse in diesem Bereich überhaupt rekonstruiert werden kann oder nicht.
    process.filter("two_lepton_mass")
    process.filter("four_lepton_mass")
    
    # # Eine weitere Verwendung von "electric_charge", können Sie die Notwendigkeit hier erklären?
    process.filter("electric_charge")
    
    # Nach der Anwendung aller Anforderungen und der impliziten Erzeugung aller notwendigen Größen, 
    # wird nun die Rekonstruktion durchgeführt. Zuerst die Z-Bosonen-Paare und dann die vier Leptonen 
    # invariante Masse aus den beiden Z-Bosonen.
    process.save(os.path.join(f"{directory}_beforeZZreco", filename))
    process.reconstruct("zz")
    process.save(os.path.join(f"{directory}_afterZZreco", filename))
    process.reconstruct("four_lepton_mass_from_zz")
    process.save(os.path.join(f"{directory}_afterHreco", filename))
    
    del process

Jetzt können Sie alle Datensätze prozessieren. Wenn Sie eine Ihrer gespeicherten Datensätze verwenden und dann die Verarbeitung von dort aus starten möchten, können Sie dies tun, indem Sie die Verzeichnisse in "collect_all_dataset_paths" entsprechend Ihrer gewählten Verzeichnisse ändern und bereits durchgeführte Schritte auskommentieren.

In [ ]:
from IPython.display import clear_output

files = collect_all_dataset_paths(["../data/for_long_analysis/mc_init/", "../data/for_long_analysis/ru_init/"])
for file in files:
    filter_and_reco_process(file=file)

# Betrachtung der finalen Verteilungen
-----------------------

Die Idee ist nun, alle Kanäle in einem Histogramm zu kombinieren und einige konkrete Variablen zu betrachten. Damit soll sichergestellt werden, dass eine ausreichende Übereinstimmung zwischen den simulierten Datensätzen und den gemessenen Datensätzen besteht. In den bisherigen Visualisierungen wurden die simulierten Datensätze in Form von Histogrammen ohne zusätzliche Skalierung dargestellt.

In diesem Abschnitt sollen die Histogramme der simulierten Datensätze so skaliert werden, dass sie der integrierten Luminosität der gemessenen Daten entsprechen, und es wird eine Zusammenführung der drei Zerfallskanäle zu einem einzigen Histogramm vorgenommen. Diese Skalierung und Zusammenführung kann für jedes Histogramm-Bin wie folgt ausgedrückt werden:

$$N_{\mathrm{bin}} = \sum_{i\in\{4\mu, 4e, 2\mu2e\}} N_{\mathrm{bin},i}\frac{\mathcal{L}_{\mathrm{exp}}\sigma_i k}{N_{\mathrm{tot},i}} \quad (*)$$

Dabei ist $N_{\mathrm{tot},i}$ die Gesamtzahl der Ereignisse aus der Monte-Carlo-Simulation des jeweiligen Kanals, $N_{\mathrm{bin},i}$ die tatsächliche Anzahl der Ereignisse in der betrachteten Histogramm-Bin und $\sigma_i$ der Wirkungsquerschnitt des jeweiligen Kanals.

Der Korrekturfaktor $k$ ($k=1$ für die Signalsimulation und $k=1.386$ für die Untergrundsimulation) ist ein für diese Simulation spezifischer Skalierungsfaktor, der nur deshalb eingeführt wurde, weil die Untergrundsimulation bis zur Präzision nächsthöherer Ordnung in der QCD (NLO) simuliert wurde. Im Gegensatz dazu erfordert die von Ihnen durchgeführte Analyse eine Genauigkeit von next-to-next-leading-order in QCD (NNLO), um alle Effekte zu berücksichtigen.

Anstelle einer neuen Simulation stellte sich heraus, dass die Einführung dieses globalen Korrekturfaktors das bestehende Problem von NLO$\rightarrow$NNLO löst.

Der $\mathcal{L}_{\mathrm{exp}}$ ist die integrierte Luminosität der verwendeten Messdaten und ist derselbe wie in der Veröffentlichung von 2012.

Um Verwirrung zu vermeiden, ist es wichtig, darauf hinzuweisen, dass diese Skalierung nicht ereignisweise erfolgt, sondern auf das gesamte Histogramm (genauer gesagt auf die Histogramme der einzelnen Kanäle) angewendet wird.

Außerdem wird hier nur die Simulation des Higgs-Boson-Signals für eine Higgs-Boson-Masse von 125 GeV verwendet, im Gegensatz zu der Veröffentlichung, die simulierte Datensätze mit anderen Massenhypothesen verwendet. Warum diese Simulation - unter Berücksichtigung der vorhandenen, veröffentlichten Messung - die geeignete ist, wird im zweiten Teil der Aufgabe erläutert.

In [ ]:
from include.histogramm.HistOf import HistOf
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 9)
h = HistOf(mc_dir="../data/for_long_analysis/mc_afterHreco",  # <- Your directory with final datasets (MC Simulation)
           ru_dir="../data/for_long_analysis/ru_afterHreco"   # <- Your directory with final datasets (Measurement)
           )

Die invariante Masse der vier Leptonen kann ähnlich wie `Apply.hist()` aufgerufen werden - der einzige Unterschied ist, dass in der Klasse `HistOf` alle Datensätze in den Verzeichnissen entsprechend skaliert wurden, bevor sie zusammengefasst werden. 

In [ ]:
h.variable("mass_4l", 37, (70, 181))
ax = plt.gca()
ax.set_xlim(70,181)
ax.set_ylim(0, 18)
ax.set_xlabel(r"$m_{4\ell}$ in GeV")
ax.set_ylabel("Entries")
plt.show()

<div class="alert alert-info">
In diesem Notebook werden die gleichen Datensätze wie in der Veröffentlichung von 2012 verwendet.    
 
* Wie groß ist in diesem Fall die betrachtete integrierte Luminosität? 
    
Das Ziel der Analyse ist die Rekonstruktion eines Higgs-Bosons, das im Vier-Lepton-Kanal in ein Paar aus einem Off-Shell- und einem On-Shell-Z-Boson zerfällt. 
    
* Was ist mit off-shell gemeint und wie ist dies motiviert? 
* Wie kann man anhand der vorhandenen Größen in den Datensätzen überprüfen, ob ein solcher Prozess erfolgreich rekonstruiert wurde? 

Im Histogramm der invarianten Massen der Leptonen ist die Messung mit einer gewissen Unsicherheit behaftet. Diese Unsicherheit erscheint sehr asymmetrisch - vor allem für niedrig besetzte Histogramm-Bins. 

* Haben Sie eine Erklärung dafür?

    
(_Hinweis_: die Messung ist ein Zählprozess)
</div>

Die möglichen Variablen, die untersucht werden können, sind:

In [ ]:
from include.processing.ApplyHelper import ProcessHelper
print(ProcessHelper.print_possible_variables("../data/for_long_analysis/mc_afterHreco/MC_2012_H_to_ZZ_to_4L_2el2mu.csv"))

Außerdem haben Sie die Möglichkeit, die Zerfallskanäle separat oder in Kombination zu untersuchen (`channel=["4e", "4mu", "2e2mu"]`), nur Ereignisse in das Histogramm aufzunehmen, die innerhalb eines bestimmten Intervalls von $m_{4\ell}$ (`"mass_4l"`) oder der Masse eines der rekonstruierten Z-Bosonen (`"z1_mass"`, `"z2_mass"`) liegen: `filter_by=["mass_4l", (115, 135)]`.

Als Beispiel: Ein Blick auf die Verteilung des Transversalimpulses der ersten beiden Leptonen im Vier-Muon- und Vier-Elektronen-Zerfallskanal, der zur Rekonstruktion der vier leptonischen invarianten Massen im Bereich von $[115, 135]$ verwendet wurde, zeigt, dass tatsächlich nur Leptonen mit einem Transversalimpuls größer oder gleich den in den Schwellenwerten eingestellten Werten verwendet wurden und dass einer der gemessenen Werte einen deutlichen Überschuss aufweist und nicht mit der Simulation übereinstimmt, wenn nur der Untergrundprozess ohne Signal berücksichtigt werden würde.

In [ ]:
h.variable("pt", bins=10, hist_range=(0,100), 
           filter_by=["mass_4l", (115, 135)], lepton_number=[0, 1], channel=["4mu", "4el"])
ax = plt.gca()
ax.set_xlabel("$p_T$ in GeV")
ax.set_ylabel("Entries")
plt.show()

In [ ]:
# Hier können Sie die Fragen von oben beantworten.
# Sie können auch die Beispiele für die oben gezeigte Aufgabe ändern.

# Schätzung der statistischen Signifikanz
--------------------

Die Idee der Bestimmung der statistischen Signifikanz werden Sie im zweiten Teil dieser Übung kennenlernen.


Eine einfache Abschätzung für die Signifikanz kann jedoch bereits hier erfolgen durch:

$$ Z = \sqrt{-2\left( s+(s+b)\ln\left( \frac{b}{s+b} \right) \right)} \, ,$$

wobei $b$ die Anzahl der Untergrundereignisse und $s$ die Anzahl der Signalereignisse ist. Die Details, wie man die obige Formel für die Signifikanz $Z$ herleitet, findet man in [**arXiv:1007.1727**](https://arxiv.org/abs/1007.1727).

<div class="alert alert-info">

* Wo in dieser Gleichung werden die gemessenen Daten implizit berücksichtigt?
* Schätzen Sie die Signifikanz des Higgs-Bosons mit der Masse von 125 GeV ab. Welche Aussagen können Sie über diesen Wert machen? 
* Leiten Sie einen Term für die Signifikanz unter der Annahme ab, dass $s\ll b$ ist, und interpretieren Sie Ihr Ergebnis. Vergleichen Sie es mit vorherigem Ergebnis.
* Ändert sich etwas, wenn Sie eine andere Anzahl von Bins verwenden oder ein anderes Massenintervall betrachten und wenn ja, warum?
</div>

In [ ]:
# necessary quantities
_, hist = h.variable("mass_4l", 15, (100, 150))
plt.show()
mc_signal, mc_background, measurement = hist.data["mc_sig"], hist.data["mc_bac"], hist.data["data"]

In [ ]:
# Ihr Code für diesen Aufgabenteil